In [1]:
!apt-get update
!apt-get install -y octave liboctave-dev
!pip install datasets huggingface_hub vllm math-verify openpyxl

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [46.9 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750 kB]
Get:13 ht

In [ ]:
from google.colab import drive

# Ejecuta la función de montaje
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!which octave
!octave --eval "disp(2 + 2)"

/usr/bin/octave
octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
4


# Definiciones de clases

In [1]:
import time
import os
# Configurar las variables de entorno para controlar el registro de vLLM
#os.environ["VLLM_CONFIGURE_LOGGING"] = "0"
#os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"
#os.environ["VLLM_USE_PROGRESS_BAR"] = "false"
import logging
from huggingface_hub import snapshot_download
from vllm import LLM, SamplingParams


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# Configurar las variables de entorno para controlar el registro de vLLM
#os.environ["VLLM_CONFIGURE_LOGGING"] = "0"
#os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"


class VLLMGenerator:
    def __init__(
        self,
        model_name: str,
        download: bool = False,
        temperature: float = 0.0,
        top_p: float = 1.0,
        max_tokens: int = 1024,
        log_prompts: bool = False
    ):
        self.model_source = model_name
        self.log_prompts = log_prompts

        if download:
            folder_name = model_name.split('/')[-1]
            logger.info("Downloading model %s to ./%s...", model_name, folder_name)
            self.model_source = snapshot_download(
                repo_id=model_name,
                local_dir=folder_name
            )
            logger.info("Model downloaded to: %s", self.model_source)

        load_start = time.time()
        self.llm = LLM(
            model=self.model_source,
            trust_remote_code=True,
            task="generate"
        )
        logger.info("Model loaded in %.2f seconds", time.time() - load_start)

        self.sampling_params = SamplingParams(
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )
        logger.info("Sampling parameters set: %s", self.sampling_params)

    def generate(self, prompts: list) -> list:
        start = time.time()
        outputs = self.llm.generate(prompts, self.sampling_params)
        logger.info("Generation completed in %.2f seconds", time.time() - start)
        return [out.outputs[0].text for out in outputs]

    def chat(self, messages: list[dict], debug: bool = None) -> str:
        prompt = ""
        for message in messages:
            role = message["role"].strip().lower()
            content = message["content"].strip()
            if role == "user":
                prompt += "<|user|>\n" + content + "\n"
            elif role == "assistant":
                prompt += "<|assistant|>\n" + content + "\n"
        prompt += "<|assistant|>\n"

        should_log = debug if debug is not None else self.log_prompts
        if should_log:
            print("📤 Prompt enviado al modelo:\n" + "-"*40 + f"\n{prompt}\n" + "-"*40)

        response = self.generate([prompt])
        return response[0].strip() if response else ""

    def batch_chat(self, conversations: list[list[dict]], debug: bool = False) -> list[str]:
        prompts = []
        for i, messages in enumerate(conversations):
            prompt = ""
            for message in messages:
                role = message["role"].strip().lower()
                content = message["content"].strip()
                if role == "user":
                    prompt += "<|user|>\n" + content + "\n"
                elif role == "assistant":
                    prompt += "<|assistant|>\n" + content + "\n"
            prompt += "<|assistant|>\n"
            prompts.append(prompt)

            if debug or self.log_prompts:
                print(f"\n📤 Prompt #{i + 1}:\n" + "-"*40 + f"\n{prompt}\n" + "-"*40)

        results = self.generate(prompts)
        return [r.strip() for r in results]


INFO 07-01 14:21:31 [__init__.py:244] Automatically detected platform cuda.


In [2]:
from typing import Dict, List

class ModelRegistry:
    """
    Registry of supported Hugging Face models for vLLMGenerator.

    Contains a fixed list of model identifiers and their metadata.
    """
    # Mapping from model key to huggingface repo id
    MODEL_REPOS: Dict[str, str] = {
        "llama3-8b-instruct": "nreHieW/Llama-3.1-8B-Instruct",
        "deepseek-math-7b": "deepseek-ai/deepseek-math-7b-instruct",
        "qwen2-7b": "Qwen/Qwen2-7B",
        "qwen2-7b-instruct": "Qwen/Qwen2-7B-Instruct",
        "qwen2-math-7b-instruct": "Qwen/Qwen2-Math-7B-Instruct",
        "mistral-7b-instruct": "mistralai/Mistral-7B-Instruct-v0.3",
        "mathstral-7b": "mistralai/Mathstral-7B-v0.1",
        "deepseek-coder-7b": "deepseek-ai/deepseek-coder-7b-instruct-v1.5",
    }

    # Additional details for each model
    MODEL_DETAILS: Dict[str, Dict[str, str]] = {
        "llama3-8b-instruct": {
            "description": "LLaMA-3 8B instruct-tuned model",
            "parameters": "8 billion",
            "context_length": "8192 tokens",
            "suitable_for": "Instruction following tasks with improved reasoning",
        },
        "deepseek-math-7b": {
            "description": "DeepSeek Math 7B instruct model",
            "parameters": "7 billion",
            "context_length": "4096 tokens",
            "suitable_for": "Mathematical problem solving and proofs",
        },
        "qwen2-7b": {
            "description": "Qwen 2 7B base model",
            "parameters": "7 billion",
            "context_length": "4096 tokens",
            "suitable_for": "General purpose text generation",
        },
        "qwen2-7b-instruct": {
            "description": "Qwen 2 7B instruct-tuned model",
            "parameters": "7 billion",
            "context_length": "4096 tokens",
            "suitable_for": "Instruction-based tasks and chat",
        },
        "qwen2-math-7b-instruct": {
            "description": "Qwen 2 Math 7B instruct model",
            "parameters": "7 billion",
            "context_length": "4096 tokens",
            "suitable_for": "Mathematical reasoning and code generation",
        },
        "mistral-7b-instruct": {
            "description": "Mistral 7B Instruct v0.3",
            "parameters": "7 billion",
            "context_length": "8192 tokens",
            "suitable_for": "Instruction following with long context",
        },
        "mathstral-7b": {
            "description": "Mathstral 7B",
            "parameters": "7 billion",
            "context_length": "8192 tokens",
            "suitable_for": "Mathematics-specific tasks",
        },
        "deepseek-coder-7b": {
            "description": "DeepSeek Coder 7B instruct model",
            "parameters": "7 billion",
            "context_length": "4096 tokens",
            "suitable_for": "Code generation and debugging",
        },
    }

    @classmethod
    def get_supported_models(cls) -> List[str]:
        """Return list of supported model keys."""
        return list(cls.MODEL_REPOS.keys())

    @classmethod
    def get_model_repo(cls, model_key: str) -> str:
        """Given a model key, return the HF repository identifier."""
        return cls.MODEL_REPOS[model_key]

    @classmethod
    def get_model_details(cls, model_key: str) -> Dict[str, str]:
        """Return metadata for a given model key."""
        return cls.MODEL_DETAILS.get(model_key, {})

    @classmethod
    def validate_model_key(cls, model_key: str) -> bool:
        """Check if a model key is registered."""
        return model_key in cls.MODEL_REPOS

    @classmethod
    def as_list_of_repos(cls) -> List[str]:
        """Return list of HF repo strings for all supported models."""
        return list(cls.MODEL_REPOS.values())

In [3]:
import subprocess
import tempfile
import os
import shutil

class OctaveExecutionError(Exception): pass
class OctaveTimeoutError(Exception): pass

class OctaveCodeExecutor:
    def __init__(self, timeout: int = 10, octave_cmd: str = 'octave', max_output_size: int = 10000):
        self.timeout = timeout
        self.octave_cmd = octave_cmd
        self.max_output_size = max_output_size

        # ✅ Verificar que el binario esté disponible
        if shutil.which(self.octave_cmd) is None:
            raise FileNotFoundError(
                f"❌ El binario '{self.octave_cmd}' no se encontró en el sistema.\n"
                f"🛠 Asegúrate de instalar Octave con:\n"
                f"!apt-get update && apt-get install -y octave"
            )

    def _write_temp_file(self, code: str) -> str:
        with tempfile.NamedTemporaryFile(mode='w', suffix='.m', delete=False) as f:
            f.write(code)
            return f.name

    def execute_with_timeout(self, code: str) -> tuple[str | None, str | None]:
        file_path = self._write_temp_file(code)
        try:
            result = subprocess.run(
                [self.octave_cmd, "--quiet", "--no-gui", "--eval", f"run('{file_path}')"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=self.timeout,
                text=True,
                check=False
            )
            stdout = result.stdout[:self.max_output_size].strip()
            stderr = result.stderr.strip()
            return stdout, stderr if stderr else None
        except subprocess.TimeoutExpired:
            raise OctaveTimeoutError("⏱️ Tiempo de ejecución excedido para el código Octave.")
        finally:
            os.remove(file_path)


In [7]:
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.1/228.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.73.0
    Uninstalling grpcio-1.73.0:
      Successfully uninstalled grpcio-1.73.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.67.1 which is incompatible.


In [8]:
from datasets import load_from_disk, Dataset
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import torch.nn.functional as F
from pymilvus import MilvusClient

# ─── Definir clase Retriever usando Milvus Lite ───────────────
class MilvusRetriever:
    def __init__(self, df, encoder, db_path, collection_name):
        self.df = df.reset_index(drop=True)
        self.encoder = encoder
        self.client = MilvusClient(uri=db_path)
        self.collection_name = collection_name

        if collection_name not in self.client.list_collections():
            self.client.create_collection(
                collection_name,
                dimension=384,
                consistency_level="Eventually",
                auto_id=True
            )
            self._index_data()
        else:
            print(f"✅ Milvus collection '{collection_name}' ya existe.")

    def _index_data(self):
        print("📥 Indexando vectores en Milvus...")
        vectors = np.stack(self.df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))).astype(np.float32)
        entities = [
            {"question": row["question"], "octave_code": row["octave_code"], "vector": vec}
            for row, vec in zip(self.df.to_dict(orient="records"), vectors)
        ]
        self.client.insert(self.collection_name, data=entities, progress_bar=True)
        print("✅ Indexación completada.")

    def retrieve(self, query, top_k=3):
        q_emb = self.encoder.encode([query], convert_to_numpy=True)
        q_emb = F.normalize(torch.tensor(q_emb), p=2, dim=1).numpy().astype(np.float32)

        results = self.client.search(
            self.collection_name,
            data=q_emb,
            output_fields=["question", "octave_code"],
            limit=top_k + 1,
            consistency_level="Eventually"
        )

        seen = set()
        retrieved = []
        for hit in results[0]:
            q_text = hit.entity["question"].strip()
            if q_text == query.strip():
                continue
            if q_text in seen:
                continue
            seen.add(q_text)
            retrieved.append(f"Problem: {q_text}\nCode: {hit.entity['octave_code']}")
            if len(retrieved) == top_k:
                break

        return "\n".join(retrieved)

    def list_all_problems(self):
        return self.df["question"].drop_duplicates().tolist()

In [9]:
import pandas as pd
import random
from collections import defaultdict

class FewShotRetriever:
    def __init__(self, csv_path: str):
        """
        Inicializa el retriever cargando el .csv y clasificando ejemplos por problem_type.

        Args:
            csv_path (str): Ruta al archivo CSV con las columnas 'question', 'octave_code' y 'problem_type'
        """
        self.examples_by_type = defaultdict(list)
        df = pd.read_csv(csv_path)

        for _, row in df.iterrows():
            problem_type = row.get("problem_type", "Unknown")
            question = row.get("question", "").strip()
            code = row.get("octave_code", "").strip()

            if question and code:
                self.examples_by_type[problem_type].append((question, code))

        print(f"✅ Indexados ejemplos por tipo de problema: {len(self.examples_by_type)} categorías")

    def retrieve(self, question_type: str, k: int = 3) -> list[str]:
        """
        Recupera k ejemplos del mismo tipo de problema.

        Args:
            question_type (str): Tipo de problema del cual recuperar ejemplos.
            k (int): Número de ejemplos a recuperar.

        Returns:
            list[str]: Lista de strings con problema + código
        """
        candidates = self.examples_by_type.get(question_type, [])
        if not candidates:
            return ""

        sampled = random.sample(candidates, min(k, len(candidates)))
        retrieved =  [f"Problem: {q}\nCode: {c}" for q, c in sampled]


        return "\n".join(retrieved)


In [10]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import pickle
import re

class BertMultiTaskClassifier(nn.Module):
    def __init__(self, dropout, num_problem_types, num_question_types):
        super(BertMultiTaskClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        for param in self.bert.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()
        self.classifier_problem = nn.Linear(768, num_problem_types)
        self.classifier_question = nn.Linear(768, num_question_types)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        pooled_output = outputs[1]
        x = self.dropout(pooled_output)
        x = self.activation(x)
        logits_problem = self.classifier_problem(x)
        logits_question = self.classifier_question(x)
        return logits_problem, logits_question

import re
import pickle

def preprocess_function(example, tokenizer, seq_len):
    text = example["problem"].lower()
    # Conservar operadores y puntuación matemática relevante
    text = re.sub(r"[^\w\d\s\+\-\*/=^√%.,()]", " ", text)
    # Normalizar repeticiones de 'x' largas
    text = re.sub(r'\b[x]{2,}\b', 'x', text)
    text = re.sub(' +', ' ', text)

    tokens = tokenizer(text, padding="max_length", max_length=seq_len, truncation=True)
    return tokens

# Funciones para guardar y cargar objetos con pickle
def save_file(name, obj):
    with open(name, 'wb') as f:
        pickle.dump(obj, f)

def load_file(name):
    return pickle.load(open(name, "rb"))

class MultiTaskInferencePipeline:
    def __init__(self, model_path, encoder_problem_path, encoder_question_path, dropout=0.5):
        self.preprocess_function = preprocess_function  # Guardamos la función

        with open(encoder_problem_path, "rb") as f:
            self.le_problem = pickle.load(f)
        with open(encoder_question_path, "rb") as f:
            self.le_question = pickle.load(f)

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
        self.model = BertMultiTaskClassifier(dropout, len(self.le_problem.classes_), len(self.le_question.classes_))
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.to(self.device)
        self.model.eval()

    def predict(self, text):
        # Preprocesar usando la misma función que durante el entrenamiento
        example = {"problem": text}
        encoded = self.preprocess_function(example, self.tokenizer, seq_len=512)

        input_ids = torch.tensor(encoded["input_ids"]).unsqueeze(0).to(self.device)
        attention_mask = torch.tensor(encoded["attention_mask"]).unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits_problem, logits_question = self.model(input_ids, attention_mask)
            pred_problem = torch.argmax(logits_problem, dim=1).cpu().numpy()[0]
            pred_question = torch.argmax(logits_question, dim=1).cpu().numpy()[0]

        label_problem = self.le_problem.inverse_transform([pred_problem])[0]
        label_question = self.le_question.inverse_transform([pred_question])[0]

        return {
            "problem_type": label_problem,
            "question_type": label_question
        }

In [22]:
import logging
from typing import Dict, List, Any
from abc import ABC, abstractmethod
import re
import json
import time
import torch.nn.functional as F
import time
from pymilvus import MilvusClient

logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

pattern_true = re.compile(r"\btrue\b", re.IGNORECASE)
pattern_false = re.compile(r"\bfalse\b", re.IGNORECASE)

class ConversationalPromptStrategy(ABC):
    """
    Clase base abstracta para estrategias conversacionales.

    Cada subclase debe implementar un flujo completo de conversación
    con el modelo, incluyendo resolubilidad, extracción de características
    (si aplica) y generación de código.
    """

    @abstractmethod
    def run_conversation(self, row: dict, generator) -> dict:
        """
        Ejecuta una conversación completa con el modelo para un problema dado.

        Args:
            row (dict): Una fila del dataset (de un CSV).
            generator: Instancia del generador (e.g., VLLMGenerator).

        Returns:
            dict: Un diccionario con al menos:
                - 'resolvability_response'
                - 'is_octave_resolvable' (bool)
                - 'problem_features' (opcional)
                - 'model_output' (código o razonamiento final)
        """
        pass





class ZeroShotConversationalStrategy(ConversationalPromptStrategy):
    def run_conversation(self, row: dict, generator) -> dict:
        start_time = time.time()  # ⏱️ Inicio del temporizador

        question = str(row.get("question", "")).strip()
        if not question:
            return {"inference_time": 0.0}

        messages = []

        # Paso 1: Introducción
        messages.append({"role": "user", "content": f"Here is a math problem:\n\n{question}"})
        messages.append({"role": "assistant", "content": "OK, I've received the problem."})

        # Paso 2: Resolvabilidad
        messages.append({
            "role": "user",
            "content": (
                "Determine if the math problem can be solved programmatically using GNU Octave."
                "Answer with 'True' or 'False' and justify your choice."
            )
        })
        resolvability_response = generator.chat(messages)
        messages.append({"role": "assistant", "content": resolvability_response})

        if re.search(pattern_true, resolvability_response) and not re.search(pattern_false, resolvability_response.split("true")[0]):
            is_octave_resolvable = True
        elif re.search(pattern_false, resolvability_response):
            is_octave_resolvable = False
        else:
            is_octave_resolvable = None  # incierto o malformado

        # Paso 3: Características del problema (si es resoluble)
        problem_features = None
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Analyze the problem and extract its key data and the following characteristics:\n"
                    "'given_data', 'unknowns', 'constraints','applicable methods','If a unit of measurement is used, specify which one'.\n\n"
                )
            })
            problem_features = generator.chat(messages)
            messages.append({"role": "assistant", "content": problem_features})

        # Paso 4: Generación de código
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Generate only a GNU Octave script that, when executed, prints the numeric answer."
                )
            })
        else:
            messages.append({
                "role": "user",
                "content": (
                   "Provide the solution to the math problem in the most suitable way."
                )
            })

        model_output = generator.chat(messages)
        messages.append({"role": "assistant", "content": model_output})

        end_time = time.time()  # ⏱️ Fin del temporizador
        inference_duration = end_time - start_time

        return {
            "resolvability_prompt": messages[0]["content"],
            "resolvability_response": resolvability_response,
            "is_octave_resolvable": is_octave_resolvable,
            "problem_features": problem_features,
            "prompt": json.dumps(messages),
            "model_output": model_output,
            "inference_time": inference_duration  # ⏱️ Tiempo total en segundos
        }

class ChainOfThoughtConversationalStrategy(ConversationalPromptStrategy):
    def run_conversation(self, row: dict, generator) -> dict:
        start_time = time.time()  # ⏱️ Inicio del temporizador

        question = str(row.get("question", "")).strip()
        rationale = str(row.get("rationale", "")).strip()
        if not question:
            return {"inference_time": 0.0}

        messages = []

        # Paso 1: Introducción
        if rationale:
            messages.append({"role": "user", "content": f"Here is a math problem:\n\n{question}.\nRationale: {rationale}\n"})
        else:
            messages.append({"role": "user", "content": f"Here is a math problem:\n\n{question}."})
        messages.append({"role": "assistant", "content": "OK, I've received the problem."})

        # Paso 2: Resolvabilidad
        messages.append({
            "role": "user",
            "content": (
                "Determine if the math problem can be solved programmatically using GNU Octave."
                "Answer with 'True' or 'False' and justify your choice."
            )
        })
        resolvability_response = generator.chat(messages)
        messages.append({"role": "assistant", "content": resolvability_response})

        if re.search(pattern_true, resolvability_response) and not re.search(pattern_false, resolvability_response.split("true")[0]):
            is_octave_resolvable = True
        elif re.search(pattern_false, resolvability_response):
            is_octave_resolvable = False
        else:
            is_octave_resolvable = None  # incierto o malformado

        # Paso 3: Características del problema (si es resoluble)
        problem_features = None
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Analyze the problem and extract its key data and the following characteristics:\n"
                    "'given_data', 'unknowns', 'constraints','applicable methods','If a unit of measurement is used, specify which one'.\n\n"
                )
            })
            problem_features = generator.chat(messages)
            messages.append({"role": "assistant", "content": problem_features})

        # Paso 4: Generación de código
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Generate only a GNU Octave script that, when executed, prints the numeric answer."
                )
            })
        else:
            messages.append({
                "role": "user",
                "content": (
                   "Provide the solution to the math problem in the most suitable way."
                )
            })

        model_output = generator.chat(messages)
        messages.append({"role": "assistant", "content": model_output})

        end_time = time.time()  # ⏱️ Fin del temporizador
        inference_duration = end_time - start_time

        return {
            "resolvability_prompt": messages[0]["content"],
            "resolvability_response": resolvability_response,
            "is_octave_resolvable": is_octave_resolvable,
            "problem_features": problem_features,
            "prompt": json.dumps(messages),
            "model_output": model_output,
            "inference_time": inference_duration  # ⏱️ Tiempo total en segundos
        }

class APEConversationalStrategy(ConversationalPromptStrategy):
    def __init__(self, rounds: int):
        """
        variants: Lista de variantes de instrucciones (plantillas) para que el modelo elija.
        """
        self.rounds = rounds

    def run_conversation(self, row: dict, generator) -> dict:
        start_time = time.time()  # ⏱️ Inicio del temporizador

        question = str(row.get("question", "")).strip()
        if not question:
            return {"inference_time": 0.0}

        messages = []

        # Paso 1: Introducción
        messages.append({"role": "user", "content": f"Here is a math problem:\n\n{question}"})
        messages.append({"role": "assistant", "content": "OK, I've received the problem."})

        # Paso 2: Resolvabilidad
        messages.append({
            "role": "user",
            "content": (
                "Determine if the math problem can be solved programmatically using GNU Octave."
                "Answer with 'True' or 'False' and justify your choice."
            )
        })
        resolvability_response = generator.chat(messages)
        messages.append({"role": "assistant", "content": resolvability_response})

        if re.search(pattern_true, resolvability_response) and not re.search(pattern_false, resolvability_response.split("true")[0]):
            is_octave_resolvable = True
        elif re.search(pattern_false, resolvability_response):
            is_octave_resolvable = False
        else:
            is_octave_resolvable = None  # incierto o malformado

        # Paso 3: Características del problema (si es resoluble)
        problem_features = None
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Analyze the problem and extract its key data and the following characteristics:\n"
                    "'given_data', 'unknowns', 'constraints','applicable methods','If a unit of measurement is used, specify which one'.\n\n"
                )
            })
            problem_features = generator.chat(messages)
            messages.append({"role": "assistant", "content": problem_features})

        # Paso 4: Generación de código
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Generate only a GNU Octave script that, when executed, prints the numeric answer."
                )
            })
        else:
            messages.append({
                "role": "user",
                "content": (
                   "Provide the solution to the math problem in the most suitable way."
                )
            })

        pattern_first = re.compile(r"\First\b", re.IGNORECASE)
        pattern_second = re.compile(r"\bSecond\b", re.IGNORECASE)

        best_variant = generator.construct_prompt(messages)
        count = 1
        while count < self.rounds:

            optimization_prompt = ( "I will provide you with a prompt that is given as input to a model whose task is to solve math problems by generating the corresponding code.\n"
                                    "Your goal is to review and improve it to make it clearer, more precise, and more effective.\n"
                                    "1. Analyze the original prompt and identify any ambiguities or missing information.\n"
                                    "2. Reformulate it to increase clarity, specificity, and include any instructions that will help the downstream model generate correct and efficient code.\n"
                                    "3. Return only the improved prompt, without any additional commentary.\n"
                                    "Here is the original prompt (between triple quotes):\n"
                                    "\"\"\"\n"
                                    f"{best_variant}"
                                    "\"\"\"\n")
            new_prompt = generator.generate([optimization_prompt])

            selection_prompt = (
            "You are given two prompt variants for solving a math problem.\n "
            "Based on the problem and context, decide which variant is the most suitable to solve it.\n"
            "Respond with 'First' if it is the first, or 'Second' if it is the second:\n"
            f"Prompt 1: {best_variant}\n"
            f"Prompt 2: {new_prompt}"
            )
            optimal = generator.generate([optimization_prompt])


            if re.search(pattern_second,optimal):
                best_variant = new_prompt
            count += 1

        model_output = generator.chat(best_variant)
        messages.append({"role": "assistant", "content": model_output})

        end_time = time.time()  # ⏱️ Fin del temporizador
        inference_duration = end_time - start_time

        return {
            "resolvability_prompt": messages[0]["content"],
            "resolvability_response": resolvability_response,
            "is_octave_resolvable": is_octave_resolvable,
            "problem_features": problem_features,
            "prompt": json.dumps(messages),
            "model_output": model_output,
            "inference_time": inference_duration  # ⏱️ Tiempo total en segundos
        }


class RAGConversationalStrategy(ConversationalPromptStrategy):
    def __init__(self, retriever):
        """
        retriever: clase con método retrieve(text: str) -> str
        """
        self.retriever = retriever

    def run_conversation(self, row: dict, generator) -> dict:
        start_time = time.time()  # ⏱️ Inicio del temporizador

        question = str(row.get("question", "")).strip()
        if not question:
            return {"inference_time": 0.0}

        messages = []

        # Paso 1: Introducción
        messages.append({"role": "user", "content": f"Here is a math problem:\n\n{question}"})
        messages.append({"role": "assistant", "content": "OK, I've received the problem."})

        # Paso 2: Resolvabilidad
        messages.append({
            "role": "user",
            "content": (
                "Determine if the math problem can be solved programmatically using GNU Octave."
                " Answer with 'True' or 'False' and justify your choice."
            )
        })
        resolvability_response = generator.chat(messages)
        messages.append({"role": "assistant", "content": resolvability_response})

        if re.search(pattern_true, resolvability_response) and not re.search(pattern_false, resolvability_response.split("true")[0]):
            is_octave_resolvable = True
        elif re.search(pattern_false, resolvability_response):
            is_octave_resolvable = False
        else:
            is_octave_resolvable = None

        # Paso 3: Características del problema (si es resoluble)
        problem_features = None
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Analyze the problem and extract its key data and the following characteristics:\n"
                    "'given_data', 'unknowns', 'constraints','applicable methods','If a unit of measurement is used, specify which one'.\n\n"
                )
            })
            problem_features = generator.chat(messages)
            messages.append({"role": "assistant", "content": problem_features})

        # Paso 4: Generación de código (con o sin RAG)
        if is_octave_resolvable:
            retrieved_context = self.retriever.retrieve(question)

            messages.append({
                "role": "user",
                "content": (
                    f"You are given related examples:\n{retrieved_context}\n\n"
                    f"Problem features: {problem_features}\n\n"
                    f"Problem: {question}\n\n"
                    "Generate only a GNU Octave script that, when executed, prints the numeric answer."
                )
            })
        else:
            messages.append({
                "role": "user",
                "content": (
                    f"Problem: {question}\n\n"
                    "Provide the solution to the math problem in the most suitable way."
                )
            })

        model_output = generator.chat(messages)
        messages.append({"role": "assistant", "content": model_output})

        end_time = time.time()  # ⏱️ Fin del temporizador
        inference_duration = end_time - start_time

        return {
            "resolvability_prompt": messages[0]["content"],
            "resolvability_response": resolvability_response,
            "is_octave_resolvable": is_octave_resolvable,
            "problem_features": problem_features,
            "prompt": json.dumps(messages),
            "model_output": model_output,
            "inference_time": inference_duration
        }

class FewShotConversationalStrategy(ConversationalPromptStrategy):
    def __init__(self, retriever, classifier=None, k=3):
        self.retriever = retriever  # debe tener método retrieve(k, problem_type=None)
        self.classifier = classifier  # puede ser None
        self.k = k

    def run_conversation(self, row: dict, generator) -> dict:
        start_time = time.time()  # ⏱️ Inicio del temporizador

        question = str(row.get("question", "")).strip()
        if not question:
            return {"inference_time": 0.0}

        messages = []

        # Paso 1: Introducción
        messages.append({"role": "user", "content": f"Here is a math problem:\n\n{question}"})
        messages.append({"role": "assistant", "content": "OK, I've received the problem."})

        # Paso 2: Resolvabilidad
        messages.append({
            "role": "user",
            "content": (
                "Determine if the math problem can be solved programmatically using GNU Octave."
                " Answer with 'True' or 'False' and justify your choice."
            )
        })
        resolvability_response = generator.chat(messages)
        messages.append({"role": "assistant", "content": resolvability_response})

        if re.search(pattern_true, resolvability_response) and not re.search(pattern_false, resolvability_response.split("true")[0]):
            is_octave_resolvable = True
        elif re.search(pattern_false, resolvability_response):
            is_octave_resolvable = False
        else:
            is_octave_resolvable = None

        # Paso 3: Características del problema (si es resoluble)
        problem_features = None
        if is_octave_resolvable:
            messages.append({
                "role": "user",
                "content": (
                    "Analyze the problem and extract its key data and the following characteristics:\n"
                    "'given_data', 'unknowns', 'constraints','applicable methods','If a unit of measurement is used, specify which one'.\n\n"
                )
            })
            problem_features = generator.chat(messages)
            messages.append({"role": "assistant", "content": problem_features})

               # Paso 4: Generación de código (con o sin RAG)
        if is_octave_resolvable:
            question_type = None
            if self.classifier:
                try:
                    preds = self.classifier.predict(question)
                    question_type = preds.get("problem_type", None)
                except:
                    question_type = None
            if question_type:
                    retrieved_context = self.retriever.retrieve(question_type = question_type, k=self.k)
            if len(retrieved_context) > 0:
                messages.append({
                    "role": "user",
                    "content": (
                        f"You are given related examples:\n{retrieved_context}\n\n"
                        f"Problem features: {problem_features}\n\n"
                        f"Problem: {question}\n\n"
                        "Generate only a GNU Octave script that, when executed, prints the numeric answer."
                    )
                })
            else:
                    messages.append({
                    "role": "user",
                    "content": (
                        f"Problem features: {problem_features}\n\n"
                        f"Problem: {question}\n\n"
                        "Generate only a GNU Octave script that, when executed, prints the numeric answer."
                    )
                })
        else:
            messages.append({
                "role": "user",
                "content": (
                    f"Problem: {question}\n\n"
                    "Provide the solution to the math problem in the most suitable way."
                )
            })


        model_output = generator.chat(messages)
        messages.append({"role": "assistant", "content": model_output})

        end_time = time.time()  # ⏱️ Fin del temporizador
        inference_duration = end_time - start_time

        return {
            "resolvability_prompt": messages[0]["content"],
            "resolvability_response": resolvability_response,
            "is_octave_resolvable": is_octave_resolvable,
            "problem_features": problem_features,
            "prompt": json.dumps(messages),
            "model_output": model_output,
            "inference_time": inference_duration
        }

In [12]:
# Rutas fijas en Google Drive para cada dataset
DATASET_PATHS = {
    "aqua": "/content/drive/MyDrive/tesis/datasets/AQUA/test.json",
    "gsm8k": "/content/drive/MyDrive/tesis/datasets/GSM-8K/test.jsonl",
    "math_data": "/content/drive/MyDrive/tesis/datasets/MATH/math_data.jsonl",
    "math_shuffled": "/content/drive/MyDrive/tesis/datasets/MATH/shuffled_math.jsonl",
    "mmlu": "/content/drive/MyDrive/tesis/datasets/MMLU/MMLU_test.jsonl",
}

In [13]:
import json
import os
import re
from typing import Dict, List, Optional

import pandas as pd


class DatasetCSVBuilder:
    """
    Transforma datasets JSON/JSONL a CSV estandarizado para el pipeline experimental,
    con soporte para procesar solo los primeros `max_items` registros si se desea.
    """

    def __init__(self, dataset_paths: Dict[str, str]):
        self.dataset_paths = dataset_paths
        self.unit_pattern = re.compile(r'[$€£%]|m\b|cm\b|kg\b|km\b')
        self.choice_pattern = re.compile(r'^([A-Za-z])\)(.*)$')

        # Columnas adicionales necesarias para el pipeline experimental completo
        self.experiment_columns = [
            "strategy",
            "model",
            "dataset",
            "resolvability_prompt",
            "resolvability_response",
            "is_octave_resolvable",
            "problem_features",
            "prompt",
            "model_output",
            "inference_time",
            "octave_code",
            "execution_output",
            "execution_error",
            "is_correct",
        ]

    # --------------------------------------------------------------------- #
    # API pública
    # --------------------------------------------------------------------- #
    def create_dataset(
        self,
        key: str, max_items: Optional[int] = None,   # ← NUEVO parámetro opcional
    ) -> pd.DataFrame:
        """
        Lee el dataset indicado por `key`, procesa hasta `max_items`
        registros (o todos si es None) y guarda el CSV resultante.

        Args:
            key: clave que identifica el dataset en `dataset_paths`.
            max_items: n.º máximo de registros a conservar (None = todos).

        Returns:
            pd.DataFrame con los datos transformados.
        """
        path = self.dataset_paths[key]
        records = self._load_records(path, max_items=max_items)
        df = pd.DataFrame(records)
        df = self._transform_dataframe(key, df)
        df = self._add_experiment_columns(df, dataset_name=key)
        self._save_as_csv(df, path, max_items=max_items)
        return df

    # --------------------------------------------------------------------- #
    # Funciones internas
    # --------------------------------------------------------------------- #
    def _load_records(
        self,
        path: str,
        *,
        max_items: Optional[int] = None,
    ) -> List[dict]:
        """
        Devuelve una lista de diccionarios cargados desde un .json o .jsonl.
        Si `max_items` está definido, corta la lista a ese tamaño.
        """
        data = []
        with open(path, "r", encoding="utf-8") as f:
            try:
                if path.endswith(".jsonl"):
                    for line in f:
                        if line.strip():
                            data.append(json.loads(line.strip()))
                            if max_items and len(data) >= max_items:
                                break
                else:  # .json o similar
                    content = json.load(f)
                    if isinstance(content, list):
                        data.extend(content[:max_items] if max_items else content)
                    else:
                        data.append(content)
            except json.JSONDecodeError as e:
                print(f"Error al leer {path}: {e}")

        return data

    def _transform_dataframe(self, name: str, df: pd.DataFrame) -> pd.DataFrame:
        # … (sin cambios: la lógica específica de cada dataset)
        if name == "aqua":
            df = df.rename(columns={"correct": "answer"})
            if "options" in df.columns:
                df["options"] = df["options"].apply(
                    lambda x: ';'.join(x) if isinstance(x, list) else x
                )
                df["answer_alternative"] = df.apply(self._map_if_unit, axis=1)

        elif name == "gsm8k":
            if "answer" in df.columns:
                df[["rationale", "answer"]] = df["answer"].str.split("####", expand=True)

        elif name in ("math_data", "math_shuffled"):
            df = df.rename(columns={"problem": "question", "solution": "rationale"})
            if "rationale" in df.columns:
                df["answer"] = df["rationale"].str.extract(r'\\boxed\{([^}]*)\}')
                df["rationale"] = df["rationale"].str.replace(
                    r'\\boxed\{([^}]*)\}', '', regex=True
                )

        elif name == "mmlu":
            df = df.rename(columns={"Question": "question"})
            options_cols = [col for col in df.columns if col.startswith("Option")]
            if options_cols:
                df["options"] = df[options_cols].apply(
                    lambda row: ';'.join(row.values.astype(str)), axis=1
                )

        return df

    def _add_experiment_columns(self, df: pd.DataFrame, dataset_name: str) -> pd.DataFrame:
        for col in self.experiment_columns:
            if col not in df.columns:
                df[col] = dataset_name if col == "dataset" else None
        return df

    def _map_if_unit(self, row):
        letter = row.get("answer")
        options = row.get("options", "")
        if not isinstance(options, str):
            return None

        options = [p.strip() for p in options.split(';') if p.strip()]
        mapping = {}
        for opt in options:
            if self.choice_pattern.match(opt):
                key, val = opt.split(')', 1)
                mapping[key] = val.strip()

        if not mapping and len(options) >= 4:
            mapping = {'A': options[0], 'B': options[1], 'C': options[2], 'D': options[3]}

        value = mapping.get(letter, None)
        return (
            self.unit_pattern.sub('', value).strip()
            if value and self.unit_pattern.search(value)
            else value
        )

    def _save_as_csv(
        self,
        df: pd.DataFrame,
        original_path: str,
        *,
        max_items: Optional[int] = None,
    ):
        """
        Guarda el CSV con un sufijo que indica si es un recorte parcial.
        Ej.: foo_experiment.csv  ó  foo_experiment_100.csv
        """
        base_dir = os.path.dirname(original_path)
        base_name = os.path.splitext(os.path.basename(original_path))[0]
        suffix = f"_experiment_{max_items}" if max_items else "_experiment"
        output_path = os.path.join(base_dir, f"{base_name}{suffix}.csv")
        df.to_csv(output_path, index=False)
        print(f"✅ CSV experimental guardado en: {output_path}")


In [14]:
MODEL_KEY = "mathstral-7b"
# Instanciar generador
generator = VLLMGenerator(
    model_name= ModelRegistry.get_model_repo(MODEL_KEY),
    download=True,
    temperature=0.3,
    top_p=0.95,
    max_tokens=1024,
    log_prompts=False
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

INFO 07-01 14:34:18 [config.py:3268] Downcasting torch.float32 to torch.bfloat16.
INFO 07-01 14:34:18 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 07-01 14:34:19 [utils.py:2597] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized


In [15]:
# ─────────────────────────────────────────────────────────────────────
# 📌 REQUISITO: Deben estar cargadas las clases del pipeline:
# - DatasetCSVBuilder, VLLMGenerator, OctaveCodeExecutor, ZeroShotConversationalStrategy
# - extract_octave_code, extract_numbers, parse, verify
# ─────────────────────────────────────────────────────────────────────

import pandas as pd
import random
import json
import os
from pathlib import Path
from datetime import datetime
from IPython.display import display, Markdown
from math_verify import parse, verify
import re

In [29]:
# ─────────────────────────────────────────────────────────────────────
# 3. Instanciar modelo y estrategia
# ─────────────────────────────────────────────────────────────────────


few_shots_retriever = FewShotRetriever(
    csv_path="/content/drive/MyDrive/tesis/datasets/NuminaMath-1.5_rag_corpus/NuminaMath-1.5_rag_corpus_final.csv"
)

classifier = MultiTaskInferencePipeline(
    model_path="/content/drive/MyDrive/tesis/categorizacion/bert_pre_trained_math_f1.pth",
    encoder_problem_path="/content/drive/MyDrive/tesis/categorizacion/problem_type_encoder.pkl",
    encoder_question_path="/content/drive/MyDrive/tesis/categorizacion/question_type_encoder.pkl"
)
strategy = FewShotConversationalStrategy(retriever=few_shots_retriever, classifier=classifier)
executor = OctaveCodeExecutor(timeout=10)

✅ Indexados ejemplos por tipo de problema: 6 categorías


In [30]:
# ─────────────────────────────────────────────────────────────────────
# 1. Cargar dataset original JSONL y convertirlo con DatasetCSVBuilder
# ─────────────────────────────────────────────────────────────────────

RAW_DATASET_PATH = "/content/drive/MyDrive/tesis/datasets/GSM-8K/test.jsonl"
TEMP_CSV_PATH = "/content/drive/MyDrive/tesis/demo/demo_gsm8k_pipeline_temp.csv"

builder = DatasetCSVBuilder(dataset_paths={"gsm8k": RAW_DATASET_PATH})
df = builder.create_dataset("gsm8k")
df["model"] = "mathstral-7b"
df["strategy"] = "few_shots"
df["dataset"] = "gsm8k"
df.to_csv(TEMP_CSV_PATH, index=False)

✅ CSV experimental guardado en: /content/drive/MyDrive/tesis/datasets/GSM-8K/test_experiment.csv


In [31]:
# ─────────────────────────────────────────────────────────────────────
# 2. Seleccionar 25 ejemplos aleatorios desde el índice 400 en adelante
# ─────────────────────────────────────────────────────────────────────

candidatos_count = 25
df = pd.read_csv(TEMP_CSV_PATH)
candidatos = df.loc[400:].copy()

if len(candidatos) < candidatos_count:
    raise ValueError("❌ No hay suficientes ejemplos desde el índice 400 en adelante.")

ejemplos_demo = candidatos.sample(candidatos_count, random_state=42).copy()
resultados = []

_CODE_RE = re.compile(r"```(?:octave|m)?[\s\r\n]*(.*?)```", re.DOTALL | re.I)
_NUM_RE = re.compile(r"[-+]?\d*\.?\d+(?:[eE][+-]?\d+)?[ij]?", re.I)

def extract_octave_code(text: str) -> str:
    m = _CODE_RE.search(str(text))
    return m.group(1).strip() if m else str(text).strip()

def extract_numbers(text: str) -> set[str]:
    return {m.group(0).lower() for m in _NUM_RE.finditer(str(text))}

for idx, row in ejemplos_demo.iterrows():
    display(Markdown(f"## ✏️ Ejemplo índice {idx}"))

    row_dict = row.to_dict()
    question = row_dict.get("question", "").strip()
    answer = str(row_dict.get("answer", "")).strip()

    display(Markdown(f"**📌 Pregunta:**\n\n```\n{question}\n```"))
    display(Markdown(f"**🎯 Respuesta esperada (gold):**\n```\n{answer}\n```"))


    try:
        result = strategy.run_conversation(row_dict, generator)

        raw_code = result.get("model_output", "")
        code = extract_octave_code(raw_code)

        stdout, stderr = executor.execute_with_timeout(code)

        if stderr:
            display(Markdown(f"**⚠️ Error en ejecución:**\n```\n{stderr}\n```"))

            correction_prompt = [
                {"role": "user", "content": f"The following Octave code caused an error:\n\n```octave\n{code}\n```\n\nError message:\n```\n{stderr}\n```\nPlease fix the code so it executes correctly."}
            ]
            fixed_code = generator.chat(correction_prompt)

            explanation_prompt = [
                {"role": "user", "content": f"Explain the purpose and logic of the following Octave code in detail:\n\n```octave\n{fixed_code}\n```"}
            ]
            explanation = generator.chat(explanation_prompt)

            stdout, stderr = executor.execute_with_timeout(extract_octave_code(fixed_code))
            code = extract_octave_code(fixed_code)
            result["model_output"] = fixed_code
            result["code_explanation"] = explanation
        else:
            explanation_prompt = [
                {"role": "user", "content": f"Explain the purpose and logic of the following Octave code in detail:\n\n```octave\n{code}\n```"}
            ]
            explanation = generator.chat(explanation_prompt)
            result["code_explanation"] = explanation

        is_correct = False
        if not stderr and stdout:
            try:
                is_correct = verify(parse(answer), parse(stdout))
            except:
                out_nums = extract_numbers(stdout)
                gold_nums = extract_numbers(answer)
                is_correct = bool(out_nums) and out_nums.issubset(gold_nums)

        display(Markdown(f"**💻 Código ejecutado:**\n```octave\n{code}\n```"))
        display(Markdown(f"**📤 Output:**\n```\n{stdout}\n```"))
        display(Markdown(f"**🧠 Explicación del código:**\n\n{explanation}"))
        display(Markdown(f"**✅ ¿Correcto?:** {'✔️ Sí' if is_correct else '❌ No'}"))
        display(Markdown("---"))

        result.update({
            "question": question,
            "answer": answer,
            "octave_code": code,
            "execution_output": stdout,
            "execution_error": stderr,
            "is_correct": is_correct
        })

        resultados.append(result)

    except Exception as e:
        display(Markdown(f"**❌ Error al procesar el ejemplo:** {e}"))

# ─────────────────────────────────────────────────────────────────────
# 5. Guardar los resultados como CSV
# ─────────────────────────────────────────────────────────────────────

final_df = pd.DataFrame(resultados)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_csv_path = f"/content/drive/MyDrive/tesis/demo/demo_resultados_pipeline_{timestamp}.csv"
final_df.to_csv(output_csv_path, index=False)

print(f"✅ Resultados guardados en: {output_csv_path}")
display(final_df[["question", "is_correct", "execution_output", "execution_error"]])


## ✏️ Ejemplo índice 472

**📌 Pregunta:**

```
Matt wants cookies for a snack, but his dad wants him to eat healthily. He tells Matt he can have half as many cookies as the number of carrot sticks he eats, plus two extra for cleaning his room. If Matt wants to eat five cookies in total, how many carrot sticks does he have to eat?
```

**🎯 Respuesta esperada (gold):**
```
6
```

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

**🧠 Explicación del código:**

This Octave code calculates the total cost of a set of kitchen items after applying a 20% discount. Here's a breakdown of the code:

1. The prices of the items are defined:
   - `pot_and_pans = 120.00`
   - `mixing_bowls = 20.00`
   - `utensils = 5 * 5.00`

   The last one is calculated as 5 times the price of 5.00, which is 25.00.

2. The total cost before the discount is calculated by adding the prices of all items:
   `total_cost_before_discount = pot_and_pans + mixing_bowls + utensils`

   This results in 120.00 + 20.00 + 25.00 = 165.00.

3. The discount is calculated as 20% of the total cost before the discount:
   `discount = total_cost_before_discount * 0.20`

   This results in 165.00 * 0.20 = 33.00.

4. The total cost after the discount is calculated by subtracting the discount from the total cost before the discount:
   `total_cost_after_discount = total_cost_before_discount - discount`

   This results in 165.00 - 33.00 = 132.00.

5. Finally, the total cost after the discount is displayed:
   `disp(total_cost_after_discount)`

   This displays 132.00.

**✅ ¿Correcto?:** ✔️ Sí

---

✅ Resultados guardados en: /content/drive/MyDrive/tesis/demo/demo_resultados_pipeline_20250701_164502.csv


,question,is_correct,execution_output,execution_error
0,"Matt wants cookies for a snack, but his dad wa...",False,,"error: 'solve' undefined near line 14, column ..."
1,Martha is knitting winter wear for her 3 grand...,True,63,None
2,Students in class 3B are collecting school poi...,True,The class is missing 145 points to go on the t...,None
3,James decides to pick some blueberries. It co...,True,10,None
4,"Although Soledad works in a windowless office,...",False,310,None
5,A charity group decides to do a yard sale. 10...,True,15,None
6,Elvis starts driving from his house and travel...,False,Elvis is 234 miles from his house.,None
7,Craig and his brother take turns spelling out ...,True,650,None
8,"If every second, a bubbling spring creates a n...",True,72000,None
9,A three-ounce box of flavored jello makes 10 s...,True,15,None
